In [74]:
#! pip install dask
import dask.dataframe as dd
import pandas as pd
import dask
import re

In [75]:
# use dask because ..
df = dd.read_csv(
    "/Users/norazhan/Desktop/Y1T1/ppol5203_ds/project/final_clean.csv",
    dtype=str,
    blocksize="64MB"
)

df.head()

,rating,title,status,pros,cons,advice,Recommend,CEO Approval,Business Outlook,Career Opportunities,Compensation and Benefits,Senior Management,Work/Life Balance,Culture & Values,Diversity & Inclusion,firm_link,date,job,index
0,5.0,Good,"Current Employee, more than 10 years",Knowledge gain of complete project,Financial growth and personal growth,<NA>,v,o,v,3,3,3,3,3.0,3.0,Reviews/Baja-Steel-and-Fence-Reviews-E5462645.htm,"Nov 19, 2022",Manager Design,<NA>
1,4.0,Good,"Former Employee, less than 1 year","Good work,good work , flexible, support","Good,work, flexible,good support, good team work",<NA>,v,o,o,4,4,4,4,4.0,4.0,Reviews/Baja-Steel-and-Fence-Reviews-E5462645.htm,"Jan 29, 2022",Anonymous Employee,<NA>
2,1.0,terrible,"Current Employee, more than 1 year",I wish there were some to list,too many to list here,<NA>,x,x,x,1.0,3.0,1.0,3.0,1.0,<NA>,https://www.glassdoor.com/Reviews/Calgary-Flam...,"Sep 24, 2020",Senior Account Executive,<NA>
3,4.0,"It could be so good, but it isn’t","Current Employee, more than 3 years",Fast Paced. Endless challenges. Inclusive envi...,The biggest perk of the job provides no value ...,<NA>,o,o,o,3.0,3.0,3.0,1.0,4.0,5.0,https://www.glassdoor.com/Reviews/Calgary-Flam...,"Mar 25, 2023",Assistant Manager,<NA>
4,3.0,Good enough,"Former Employee, more than 5 years","Discounts, benefits, life long friends.","Drama, playing favorites, low pay",<NA>,x,r,r,2.0,2.0,2.0,2.0,2.0,2.0,https://www.glassdoor.com/Reviews/Calgary-Flam...,"Apr 6, 2023",Floor Supervisor,<NA>


In [76]:
import dask.dataframe as dd

# ================================================================
# 1) CLEAN COLUMN NAMES (lowercase and snake_case)
# ================================================================
df.columns = (
    df.columns.str.lower()
              .str.strip()
              .str.replace(" ", "_")
              .str.replace("/", "_")
              .str.replace("&", "and")
)

# ================================================================
# 2) FIX v / x / o COLUMNS — CLEAN BEFORE ANY GLOBAL REPLACEMENT
# ================================================================
vx_cols = ["recommend", "ceo_approval", "business_outlook"]

vx_map = {"v": 1, "x": -1, "o": 0}

for col in vx_cols:
    if col in df.columns:
        df[col] = df[col].str.strip().str.lower()  # safe cleaning
        df[col] = df[col].map(vx_map).astype("float32")

# ================================================================
# 3) CONVERT NUMERIC RATING COLUMNS
# ================================================================
numeric_cols = [
    "rating",
    "career_opportunities",
    "compensation_and_benefits",
    "senior_management",
    "work_life_balance",
    "culture_and_values",
    "diversity_and_inclusion"
]

for col in numeric_cols:
    if col in df.columns:
        df[col] = dd.to_numeric(df[col], errors="coerce")

# ================================================================
# 4) CLEAN SPECIAL STRINGS (SAFE NOW)
# Only remove real placeholders — NOT whitespace.
# ================================================================
df = df.replace({"<na>": None})

# ================================================================
# 5) OPTIONAL: PARSE DATES
# ================================================================
if "date" in df.columns:
    df["date"] = dd.to_datetime(df["date"], errors="coerce")

# ================================================================
# 6) PREVIEW
# ================================================================
df.head()


,rating,title,status,pros,cons,advice,recommend,ceo_approval,business_outlook,career_opportunities,compensation_and_benefits,senior_management,work_life_balance,culture_and_values,diversity_and_inclusion,firm_link,date,job,index
0,5.0,Good,"Current Employee, more than 10 years",Knowledge gain of complete project,Financial growth and personal growth,<NA>,1.0,0.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,Reviews/Baja-Steel-and-Fence-Reviews-E5462645.htm,2022-11-19,Manager Design,<NA>
1,4.0,Good,"Former Employee, less than 1 year","Good work,good work , flexible, support","Good,work, flexible,good support, good team work",<NA>,1.0,0.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,Reviews/Baja-Steel-and-Fence-Reviews-E5462645.htm,2022-01-29,Anonymous Employee,<NA>
2,1.0,terrible,"Current Employee, more than 1 year",I wish there were some to list,too many to list here,<NA>,-1.0,-1.0,-1.0,1.0,3.0,1.0,3.0,1.0,<NA>,https://www.glassdoor.com/Reviews/Calgary-Flam...,2020-09-24,Senior Account Executive,<NA>
3,4.0,"It could be so good, but it isn’t","Current Employee, more than 3 years",Fast Paced. Endless challenges. Inclusive envi...,The biggest perk of the job provides no value ...,<NA>,0.0,0.0,0.0,3.0,3.0,3.0,1.0,4.0,5.0,https://www.glassdoor.com/Reviews/Calgary-Flam...,2023-03-25,Assistant Manager,<NA>
4,3.0,Good enough,"Former Employee, more than 5 years","Discounts, benefits, life long friends.","Drama, playing favorites, low pay",<NA>,-1.0,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,https://www.glassdoor.com/Reviews/Calgary-Flam...,2023-04-06,Floor Supervisor,<NA>


In [77]:
target_companies = {
    # Top 15 healthcare
    "Mayo Clinic": "E19884",
    "Kaiser Permanente": "E19466",
    "Cleveland Clinic": "E17787",
    "Johns Hopkins Health System": "E19887",
    "HCA Healthcare": "E2062",
    "UnitedHealth Group": "E1991",
    "CVS Health": "E437",
    "Humana": "E340",
    "Cigna": "E119",
    "Tenet Healthcare": "E466",
    "Boston Scientific": "E2187",
    "Pfizer": "E525",
    "Johnson & Johnson": "E364",
    "Roche": "E3480",
    "GSK": "E3477",
    # Top 15 infrastructure
    "Bechtel": "E2731",
    "Jacobs": "E913",
    "AECOM": "E5632",
    "Fluor": "E262",
    "Kiewit Corporation": "E2935",
    "Skanska": "E10372",
    "Turner Construction": "E19014",
    "Vinci": "E10584",
    "Balfour Beatty": "E10135",
    "AtkinsRéalis": "E8810",
    "DPR Construction": "E3157",
    "Arcadis": "E32491",
    "Parsons Corporation": "E4006",
    "HDR": "E15172",
    "WSP": "E13584"
}

In [78]:
df["firm_id"] = df["firm_link"].str.extract(r"(E\d+)", expand=False)
df["firm_id"].dropna().head(20)

0     E5462645
1     E5462645
2        E5247
3        E5247
4        E5247
5        E5247
6        E5247
7        E5247
8        E5247
9        E5247
10       E5247
11       E5247
12       E5247
13       E5247
14       E5247
15       E5247
16       E5247
17       E5247
18       E5247
19       E5247
Name: firm_id, dtype: string

In [80]:
# 3. Compute how many reviews match each company
# compute firm_id counts once
firm_counts = df["firm_id"].value_counts().compute()

In [81]:
results = {
    name: int(firm_counts.get(eid, 0))
    for name, eid in target_companies.items()
}

In [87]:
results

{'Mayo Clinic': 5274,
 'Kaiser Permanente': 0,
 'Cleveland Clinic': 3467,
 'Johns Hopkins Health System': 0,
 'HCA Healthcare': 5326,
 'UnitedHealth Group': 0,
 'CVS Health': 26232,
 'Humana': 5124,
 'Cigna': 4099,
 'Tenet Healthcare': 1167,
 'Boston Scientific': 0,
 'Pfizer': 5198,
 'Johnson & Johnson': 8102,
 'Roche': 0,
 'GSK': 6788,
 'Bechtel': 0,
 'Jacobs': 4637,
 'AECOM': 0,
 'Fluor': 2109,
 'Kiewit Corporation': 0,
 'Skanska': 371,
 'Turner Construction': 0,
 'Vinci': 0,
 'Balfour Beatty': 0,
 'AtkinsRéalis': 1163,
 'DPR Construction': 447,
 'Arcadis': 2311,
 'Parsons Corporation': 1301,
 'HDR': 1093,
 'WSP': 3302}

In [85]:
matching_eids = [eid for eid in target_companies.values() if firm_counts.get(eid, 0) > 0]
matching_eids

['E19884',
 'E17787',
 'E2062',
 'E437',
 'E340',
 'E119',
 'E466',
 'E525',
 'E364',
 'E3477',
 'E913',
 'E262',
 'E10372',
 'E8810',
 'E3157',
 'E32491',
 'E4006',
 'E15172',
 'E13584']

In [86]:
filtered_df = df[df["firm_id"].isin(matching_eids)]
pdf = filtered_df.compute()     # loads only filtered rows into memory
pdf.head()

,rating,title,status,pros,cons,advice,recommend,ceo_approval,business_outlook,career_opportunities,compensation_and_benefits,senior_management,work_life_balance,culture_and_values,diversity_and_inclusion,firm_link,date,job,index,firm_id
152602,5.0,Overall Good Oil and Gas Company,Former Employee,I love the corporate culture,The outsourcing for IT didn't work,<NA>,0.0,0.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,https://www.glassdoor.com/Reviews/SNC-Lavalin-...,2023-01-25,IT Technician,<NA>,E8810
152603,3.0,Very easy work not really learning,"Former Employee, less than 1 year",Good pay reasonable work and near transit,hard to find work and not helpful at learning ...,<NA>,1.0,NaN,NaN,2.0,4.0,3.0,3.0,3.0,5.0,https://www.glassdoor.com/Reviews/SNC-Lavalin-...,2023-03-28,Mechanical Engineer,<NA>,E8810
152604,3.0,Great work,Current Employee,"Good benefits, good pay, good work culture",Work life balance is not the best,<NA>,0.0,0.0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,https://www.glassdoor.com/Reviews/SNC-Lavalin-...,2023-01-23,Business Analyst,<NA>,E8810
152605,2.0,Field Team,Former Employee,"Variety in the day, opportunity to take on sev...","Long days, mediocre pay, frequent lay offs",<NA>,0.0,0.0,0.0,2.0,2.0,2.0,1.0,1.0,1.0,https://www.glassdoor.com/Reviews/SNC-Lavalin-...,2023-01-09,Field Purchasing Manager,<NA>,E8810
152606,4.0,Good,"Current Employee, more than 3 years",Good work culture Good coworkers,Not any cons I can think of,<NA>,1.0,NaN,NaN,5.0,4.0,5.0,5.0,4.0,5.0,https://www.glassdoor.com/Reviews/SNC-Lavalin-...,2023-01-11,Designer,<NA>,E8810


In [72]:
pdf.shape

(87511, 20)

In [89]:
unique_ids = pdf["firm_link"].unique()
unique_ids

<ArrowStringArray>
[                            'https://www.glassdoor.com/Reviews/SNC-Lavalin-Reviews-E8810.htm',
                                'https://www.glassdoor.com/Reviews/Arcadis-Reviews-E32491.htm',
                                                             'Reviews/Humana-Reviews-E340.htm',
                        'https://www.glassdoor.com/Reviews/DPR-Construction-Reviews-E3157.htm',
                                                    'Reviews/HCA-Healthcare-Reviews-E2062.htm',
                                                         'Reviews/CVS-Health-Reviews-E437.htm',
 'https://www.glassdoor.com/Reviews/CVS-Health-Reviews-E437_P2019.htm?filter.iso3Language=eng',
                                                  'Reviews/Johnson-&-Johnson-Reviews-E364.htm',
                                                   'Reviews/Tenet-Healthcare-Reviews-E466.htm',
                     'https://www.glassdoor.com/Reviews/Parsons-Corporation-Reviews-E4006.htm',
                     

In [90]:
mapping = pdf.groupby("firm_id")["firm_link"].unique()

for fid, links in mapping.items():
    print("\n====", fid, "====")
    for link in links:
        print(link)


==== E10372 ====
https://www.glassdoor.com/Reviews/Skanska-Reviews-E10372.htm

==== E119 ====
Reviews/The-Cigna-Group-Reviews-E119.htm

==== E13584 ====
https://www.glassdoor.com/Reviews/WSP-Reviews-E13584.htm

==== E15172 ====
https://www.glassdoor.com/Reviews/HDR-Reviews-E15172.htm

==== E17787 ====
https://www.glassdoor.com/Reviews/Cleveland-Clinic-Reviews-E17787.htm

==== E19884 ====
https://www.glassdoor.com/Reviews/Mayo-Clinic-Reviews-E19884.htm

==== E2062 ====
Reviews/HCA-Healthcare-Reviews-E2062.htm

==== E262 ====
Reviews/Fluor-Reviews-E262.htm

==== E3157 ====
https://www.glassdoor.com/Reviews/DPR-Construction-Reviews-E3157.htm

==== E32491 ====
https://www.glassdoor.com/Reviews/Arcadis-Reviews-E32491.htm

==== E340 ====
Reviews/Humana-Reviews-E340.htm

==== E3477 ====
https://www.glassdoor.com/Reviews/GSK-Reviews-E3477.htm

==== E364 ====
Reviews/Johnson-&-Johnson-Reviews-E364.htm

==== E4006 ====
https://www.glassdoor.com/Reviews/Parsons-Corporation-Reviews-E4006.htm

===

Split based on firm_id, not firm_link

In [91]:
# --- define firm groups ---
healthcare_firms = {
    "Mayo Clinic", "Kaiser Permanente", "Cleveland Clinic",
    "Johns Hopkins Health System", "HCA Healthcare",
    "UnitedHealth Group", "CVS Health", "Humana", "Cigna",
    "Tenet Healthcare", "Pfizer", "Johnson & Johnson", "Roche", "GSK"
}

infrastructure_firms = {
    "Bechtel", "Jacobs", "AECOM", "Fluor", "Kiewit Corporation",
    "Skanska", "Turner Construction", "Vinci", "Balfour Beatty",
    "AtkinsRéalis", "DPR Construction", "Arcadis", 
    "Parsons Corporation", "HDR", "WSP"
}

# map company names to firm_ids
healthcare_eids = [target_companies[name] for name in healthcare_firms]
infra_eids = [target_companies[name] for name in infrastructure_firms]

# split based on firm_id (CORRECT)
healthcare_df = pdf[pdf["firm_id"].isin(healthcare_eids)]
infrastructure_df = pdf[pdf["firm_id"].isin(infra_eids)]

# show shapes
healthcare_df.shape, infrastructure_df.shape

((70777, 20), (16734, 20))

In [92]:
# --- save the two split DataFrames ---
healthcare_df.to_csv("healthcare_reviews.csv", index=False)
infrastructure_df.to_csv("infrastructure_reviews.csv", index=False)